In [1]:
import sys
from pathlib import Path
import os
import json

sys.path.append(str(Path.cwd().parent.parent))
print(str(Path.cwd().parent.parent))
print("Current working directory: ", os.getcwd())

from ModelDevelopment.CentralPipeline import CentralPipeline
from ModelDevelopment.ImageBatchPipeline import ImageBatchPipeline
from DataProcessing.DataPreProcessing import DataPaths

c:\Users\colin\OneDrive\Desktop\Jersey-Number-Recognition
Current working directory:  c:\Users\colin\OneDrive\Desktop\Jersey-Number-Recognition\ModelDevelopment\experiments


In [2]:
gt_test = DataPaths.TEST_DATA_GT.value
#Schema of gt_test: {"0": 4, "1": 93, "2": 25, "3": -1, "4": -1,

# load the json and compile a list of all tracklets with -1 as not_legible and all other tracklets as legible
with open(gt_test, 'r') as f:
    data = json.load(f)
    legible = []
    illegible = []
    
    for key, value in data.items():
        if value == -1:
            illegible.append(key)
        else:
            legible.append(key)
    
print("Legible tracklets: ", legible)
print("Illegible tracklets: ", illegible)

Legible tracklets:  ['0', '1', '2', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '26', '27', '28', '30', '31', '32', '33', '34', '35', '36', '37', '42', '43', '44', '45', '46', '48', '52', '54', '55', '56', '57', '58', '59', '60', '62', '63', '64', '65', '66', '67', '68', '69', '70', '72', '73', '77', '80', '83', '84', '86', '88', '91', '92', '93', '94', '95', '96', '98', '99', '100', '101', '102', '103', '105', '106', '107', '109', '111', '112', '113', '114', '115', '116', '120', '123', '125', '126', '127', '128', '131', '132', '133', '135', '136', '137', '138', '139', '140', '141', '142', '143', '144', '145', '146', '148', '149', '150', '151', '152', '153', '154', '156', '157', '159', '160', '161', '163', '166', '169', '170', '171', '173', '178', '179', '180', '181', '185', '186', '187', '188', '189', '190', '191', '192', '193', '196', '197', '198', '201', '202', '203', '204', '205', '206', '207', '208', '209', '210', '212', '213', '214', '219'

In [3]:
# # Delete soccer_ball.json files
# import os
# root_path = r"C:\Users\colin\OneDrive\Desktop\Jersey-Number-Recognition\data\SoccerNet\jersey-2023\extracted\test\images"

# for tracklet in range(1, 1210):
#   path = os.path.join(root_path, str(tracklet))
  
#   # If there is a .soccer_ball.json file in there, delete it
#   if os.path.exists(os.path.join(path, "soccer_ball.json")):
#     os.remove(os.path.join(path, "soccer_ball.json"))

In [4]:
pipeline = CentralPipeline(
  #tracklets_to_process_override=[109, 88, 34, 151, 19, 68, 67, 128, 57, 178, 189, 196, 214, 242, 258, 271, 304, 327, 320, 352, 375, 378, 414, 416, 439, 425, 451, 453, 454, 466, 499, 511, 513, 526, 528, 567, 576, 594, 619, 623, 646, 687, 682, 709, 711, 717, 739, 758, 767, 795, 788, 840, 843, 832, 841, 852, 847, 851, 856, 850, 868, 866, 871, 901, 902, 904, 900, 907, 928, 939, 944, 961, 984, 983, 1005, 1039, 1064, 1063, 1077, 1091, 1101, 1137, 1152, 1153, 1160, 1167, 1157, 1165, 1206],
  tracklets_to_process_override=[327, 711, 907, 1064], # These ones are now passing 
  #tracklets_to_process_override=illegible,
  #tracklets_to_process_override=[168, 167, 162, 97, 39, 87, 89, 134, 61, 172, 164, 199, 215, 260, 267, 269, 281, 276, 288, 295, 317, 325, 340, 345, 374, 394, 388, 386, 391, 406, 408, 434, 437, 459, 471, 514, 532, 558, 683, 748, 770, 790, 802, 827, 861, 869, 879, 897, 895, 909, 964, 981, 999, 1023, 1031, 1025, 1033, 1030, 1055, 1088, 1087, 1169, 1175, 1174, 1173, 1183, 1170, 1179, 1184, 1185],
  #num_tracklets=1,
  #num_images_per_tracklet=50,
  input_data_path=DataPaths.TEST_DATA_DIR.value,
  output_processed_data_path=DataPaths.PROCESSED_DATA_OUTPUT_DIR_TEST.value,
  common_processed_data_dir=DataPaths.COMMON_PROCESSED_OUTPUT_DATA_TEST.value,
  gt_data_path=DataPaths.TEST_DATA_GT.value,
  display_transformed_image_sample=False, # NOTE: DO NOT USE. Code is parallelized so we cannot show images anymore. Code breaks, but first one will show if True.
  use_cache=False, # Set to false to rebuild the cache
  suppress_logging=False,
  
  # --- PARALLELIZATION PARAMS --- These settings are optimal for an NVIDIA RTX 3070 Ti Laptop GPU.
  num_workers=2,            # CRITICAL optimisation param. Adjust accordingly. Pre-processing: 4 | STR: 2 | Crops: 4+
  tracklet_batch_size=32,   # CRITICAL optimisation param. Adjust accordingly. Always 32
  image_batch_size=100,     # CRITICAL optimisation param. Adjust accordingly. Pre-processing: 200 | STR: 100 | Crops: 200
  num_threads_multiplier=2  # CRITICAL optimisation param. Adjust accordingly. Pre-processing: 4
  )                         #                              Interpretation: num_threads = num_workers * num_threads_multiplier

2025-03-30 15:07:25 [INFO] DataPreProcessing initialized. Universe of available data paths:
2025-03-30 15:07:25 [INFO] ROOT_DATA_DIR: c:\Users\colin\OneDrive\Desktop\Jersey-Number-Recognition\data\SoccerNet\jersey-2023\extracted
2025-03-30 15:07:25 [INFO] TEST_DATA_GT: c:\Users\colin\OneDrive\Desktop\Jersey-Number-Recognition\data\SoccerNet\jersey-2023\extracted\test\test_gt.json
2025-03-30 15:07:25 [INFO] TRAIN_DATA_GT: c:\Users\colin\OneDrive\Desktop\Jersey-Number-Recognition\data\SoccerNet\jersey-2023\extracted\train\train_gt.json
2025-03-30 15:07:25 [INFO] TEST_DATA_DIR: c:\Users\colin\OneDrive\Desktop\Jersey-Number-Recognition\data\SoccerNet\jersey-2023\extracted\test\images
2025-03-30 15:07:25 [INFO] TRAIN_DATA_DIR: c:\Users\colin\OneDrive\Desktop\Jersey-Number-Recognition\data\SoccerNet\jersey-2023\extracted\train\images
2025-03-30 15:07:25 [INFO] CHALLENGE_DATA_DIR: c:\Users\colin\OneDrive\Desktop\Jersey-Number-Recognition\data\SoccerNet\jersey-2023\extracted\challenge\images
2

In [5]:
pipeline.run_soccernet(
  run_soccer_ball_filter=False,
  generate_features=False,
  run_filter=False,
  run_legible=False,
  run_legible_eval=False,
  run_pose=False,
  run_crops=False,
  run_str=True,
  run_combine=False,
  run_eval=False,
  use_clip4str=False,
  pyscript=False
  )

2025-03-30 15:07:25 [INFO] Running the SoccerNet pipeline.
2025-03-30 15:07:25 [INFO] Using PARSEQ2 model for scene text recognition
2025-03-30 15:07:25 [INFO] Predicting numbers
2025-03-30 15:07:25 [INFO] Aggregating legible & illegible results (cache not used or only one file is missing).
2025-03-30 15:07:25 [INFO] Saved global legible results to: c:\Users\colin\OneDrive\Desktop\Jersey-Number-Recognition\data\SoccerNet\jersey-2023\processed_data\test\common_data\legible_results.json
2025-03-30 15:07:25 [INFO] Saved global illegible results to: c:\Users\colin\OneDrive\Desktop\Jersey-Number-Recognition\data\SoccerNet\jersey-2023\processed_data\test\common_data\illegible_results.json
2025-03-30 15:07:25 [INFO] Legible tracklets list: 327, 711, 1064
Current working directory:  c:\Users\colin\OneDrive\Desktop\Jersey-Number-Recognition


Dispatching STR on tracklets:   0%|          | 0/3 [00:00<?, ?it/s]

Aggregating STR results:   0%|          | 0/3 [00:00<?, ?it/s]

2025-03-30 15:07:37 [INFO] Tracklet 711 stdout: Additional keyword arguments: {'charset_test': '0123456789'}
STR Inference:   0%|          | 0/2 [00:00<?, ?it/s]
Results saved to c:\Users\colin\OneDrive\Desktop\Jersey-Number-Recognition\data\SoccerNet\jersey-2023\processed_data\test\711\str_results.json


2025-03-30 15:07:37 [ERROR] Tracklet 711 stderr: C:\Users\colin\miniconda3\envs\parseq2\lib\site-packages\timm\models\helpers.py:7: FutureWarning: Importing from timm.models.helpers is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)
C:\Users\colin\miniconda3\envs\parseq2\lib\site-packages\lightning_fabric\utilities\cloud_io.py:51: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https:/

Merging STR results:   0%|          | 0/4 [00:00<?, ?it/s]

2025-03-30 15:07:38 [INFO] Raw loaded data for tracklet 907: {}
2025-03-30 15:07:38 [INFO] Raw loaded data for tracklet 1064: {'1064_226.jpg': {'label': '29', 'confidence': [0.8989174962043762, 0.9955570101737976, 1.0], 'raw': [[2.820311628681793e-08, 1.0250173687609276e-07, 0.10106583684682846, 0.8989174962043762, 1.4647814168711193e-05, 3.141818751828396e-07, 6.092017912351366e-08, 5.270389902278794e-09, 9.849470217204725e-08, 7.929074854473583e-07, 7.184305559349013e-07], [1.3559057379097794e-06, 0.0013297226978465915, 3.7749330658698454e-05, 3.614215984271141e-07, 7.329044251491723e-07, 1.4522301171382423e-05, 0.0009881602600216866, 3.053697582799941e-05, 0.00045780595974065363, 0.001582057448104024, 0.9955570101737976], [1.0, 8.926825945110295e-10, 5.080534881685139e-10, 2.395583557912584e-11, 1.4357225130989804e-10, 3.4238770672823104e-12, 7.897747560103863e-12, 2.9892878537080003e-12, 3.933409153944467e-12, 1.774560316400442e-12, 2.6873598832355405e-12]], 'logits': [[-1.03369331

Merging STR results: 100%|██████████| 4/4 [00:00<00:00, 626.97it/s]

2025-03-30 15:07:38 [INFO] Saved global STR results to: c:\Users\colin\OneDrive\Desktop\Jersey-Number-Recognition\data\SoccerNet\jersey-2023\processed_data\test\common_data\str_results.json


## Benchmark (current pipeline metrics)
### Results
- 2025-03-29 14:44:05 [INFO] Correct 1051 out of 1210. Accuracy 86.85950413223141%.
- 2025-03-29 14:44:05 [INFO] TP=766, TN=285, FP=70, FN=89
- 2025-03-29 14:44:05 [INFO] Precision=0.916267942583732, Recall=0.895906432748538
- 2025-03-29 14:44:05 [INFO] F1=0.9059727971614429
- 2025-03-29 14:44:05 [INFO] Saved failed legibility cases to:  c:\Users\colin\OneDrive\Desktop\Jersey-Number-Recognition\data\SoccerNet\jersey-2023\processed_data\test\common_data\failed_legibility_cases.json

### Analysis
legible_but_marked_illegible: 89 cases
109, 88, 34, 151, 19, 68, 67, 128, 57, 178, 189, 196, 214, 242, 258, 271, 304, 327, 320, 352, 375, 378, 414, 416, 439, 425, 451, 453, 454, 466, 499, 511, 513, 526, 528, 567, 576, 594, 619, 623, 646, 687, 682, 709, 711, 717, 739, 758, 767, 795, 788, 840, 843, 832, 841, 852, 847, 851, 856, 850, 868, 866, 871, 901, 902, 904, 900, 907, 928, 939, 944, 961, 984, 983, 1005, 1039, 1064, 1063, 1077, 1091, 1101, 1137, 1152, 1153, 1160, 1167, 1157, 1165, 1206

illegible_but_marked_legible: 70 cases
168, 167, 162, 97, 39, 87, 89, 134, 61, 172, 164, 199, 215, 260, 267, 269, 281, 276, 288, 295, 317, 325, 340, 345, 374, 394, 388, 386, 391, 406, 408, 434, 437, 459, 471, 514, 532, 558, 683, 748, 770, 790, 802, 827, 861, 869, 879, 897, 895, 909, 964, 981, 999, 1023, 1031, 1025, 1033, 1030, 1055, 1088, 1087, 1169, 1175, 1174, 1173, 1183, 1170, 1179, 1184, 1185

- Total misclassifications: 159
- False negatives (legible marked as illegible): 89
- False positives (illegible marked as legible): 70
- Percentage of false negatives: 7.36%
- Percentage of false positives: 5.79%
- Combined percentage of errors as a percentage of total cases: 13.14%

In [6]:
import json
failed_cases = f"{DataPaths.COMMON_PROCESSED_OUTPUT_DATA_TEST.value}/failed_legibility_cases.json"

# Load this JSON file
with open(failed_cases, 'r') as f:
    failed_cases_dict = json.load(f)
    # Print the every key and its associated value (value will be a list so need to make that printable with ",".join):
    for key, value in failed_cases_dict.items():
        print(f"{key}: {len(value)} cases")
        if len(value) > 0:
            print(f"{', '.join(value)}")
        print()
    
    # Summary statistics
    legible_but_marked_illegible = len(failed_cases_dict.get("legible_but_marked_illegible", []))
    illegible_but_marked_legible = len(failed_cases_dict.get("illegible_but_marked_legible", []))
    total_errors = legible_but_marked_illegible + illegible_but_marked_legible
    
    print(f"Total misclassifications: {total_errors}")
    print(f"False negatives (legible marked as illegible): {legible_but_marked_illegible}")
    print(f"False positives (illegible marked as legible): {illegible_but_marked_legible}")
    print(f"Percentage of false negatives: {legible_but_marked_illegible / 1210 * 100:.2f}%")
    print(f"Percentage of false positives: {illegible_but_marked_legible / 1210 * 100:.2f}%")
    print(f"Combined percentage of errors as a percentage of total cases: {total_errors / 1210 * 100:.2f}%")

legible_but_marked_illegible: 1 cases
907

illegible_but_marked_legible: 0 cases

Total misclassifications: 1
False negatives (legible marked as illegible): 1
False positives (illegible marked as legible): 0
Percentage of false negatives: 0.08%
Percentage of false positives: 0.00%
Combined percentage of errors as a percentage of total cases: 0.08%


In [7]:
# import timm
# import torch
# from torch import nn

# model = timm.create_model("timm/vit_base_patch16_224.orig_in21k_ft_in1k", pretrained=False)
# model.head = nn.Linear(model.head.in_features, 10)
# model.load_state_dict(
#     torch.hub.load_state_dict_from_url(
#         "https://huggingface.co/edadaltocg/vit_base_patch16_224_in21k_ft_svhn/resolve/main/pytorch_model.bin",
#         map_location="cpu",
#         file_name="vit_base_patch16_224_in21k_ft_svhn.pth",
#     )
# )